<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f0e664783ea5781a832327eaafe39848ed3d347521eaad572aeba4c88202c6b9
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 15:17:19
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 825.00 (0.01%)
Current PnL: -16.76 L (-11.27%)
CY Booked + Current PnL: -5.32 L (-3.58%)
-------------------
Total profit:  2.18 L
Total loss:  -18.94 L
-------------------
Total Booked + Current PnL: 21.66 L (17.8%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.27 L (56.89%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.49,196340.0,31670.0,11663.0,0.12,19.23,5.94,26.31,89.0,2.72,1.44,28.75,X40N,NTT,AC
76,TTKPRESTIG,770.00,101.71,50.0,M-SC,3.72,86866.0,-13911.0,14003.0,-1.56,-13.80,16.12,0.09,245.0,-0.99,0.64,14.12,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,213045.0,21303.0,16447.0,-0.41,11.11,7.72,19.69,99.0,1.30,1.56,18.66,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,66.0,H-MC,7.96,112434.0,11930.0,17180.0,0.40,11.87,15.28,28.96,79.0,0.69,0.82,49.87,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219632.0,9266.0,19525.0,-0.50,4.40,8.89,13.68,37.0,0.47,1.61,21.66,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-14.58,78.0,H-SC,12.46,228613.0,-42148.0,372091.0,6.79,-15.57,162.76,121.86,133.0,-0.11,1.67,78.84,AR,ATH,IT
51,PGHH,17973.08,-28.56,65.0,H-MC,4.04,206865.0,6045.0,62721.0,3.28,3.01,30.32,34.25,80.0,0.10,1.51,8.36,X40,ATH,FMCG
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,85968.0,-16242.0,109635.0,2.94,-15.89,127.53,91.37,208.0,-0.15,0.63,39.55,XY25,NTT,FINANCE
78,UNIONBANK,163.00,-11.63,67.0,M-LC,7.07,159680.0,18840.0,25645.0,2.82,13.38,16.06,31.59,66.0,0.73,1.17,43.41,XY24,NTT,BANKS
59,SAIL,228.00,48.36,71.0,M-MC,7.57,234536.0,9574.0,158077.0,2.27,4.26,67.40,74.53,192.0,0.06,1.72,37.88,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RELAXO,1176.00,-35.33,45.0,H-SC,1.81,88144.0,-57016.0,132948.0,-5.36,-39.28,150.83,52.31,136.0,-0.43,0.64,17.99,X40N,NTT,FOOTWEAR
80,VAIBHAVGBL,521.00,60.88,63.0,H-SC,6.65,139443.0,-43332.0,154405.0,-2.93,-23.71,110.73,60.77,125.0,-0.28,1.02,24.89,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3797.30,59.0,M-SC,5.38,121446.0,-25922.0,77738.0,-2.57,-17.59,64.01,35.16,236.0,-0.33,0.89,24.04,XY24,NTT,AUTO
58,ROUTE,2236.84,-47.48,42.0,H-SC,17.55,86274.0,-46792.0,139643.0,-2.42,-35.16,161.86,69.78,118.0,-0.34,0.63,0.74,SR,ATH,IT
62,SFL,1287.00,32.67,39.0,M-SC,8.00,175143.0,-87075.0,156911.0,-2.34,-33.21,89.59,26.63,229.0,-0.55,1.28,8.89,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-20.54,46.0,H-MC,2.22,136730.0,942.0,41812.0,0.82,0.69,30.58,31.48,107.0,0.02,1.00,12.90,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.91,248087.0,356.0,74699.0,-0.25,0.14,30.11,30.30,92.0,0.00,1.82,13.45,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,96.03,56.0,H-SC,9.43,124011.0,-18468.0,37985.0,0.92,-12.96,30.63,13.70,163.0,-0.49,0.91,46.37,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,101.71,50.0,M-SC,3.72,86866.0,-13911.0,14003.0,-1.56,-13.80,16.12,0.09,245.0,-0.99,0.64,14.12,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,61.0,H-SC,1.27,226341.0,-43326.0,79649.0,-0.69,-16.07,35.19,13.47,138.0,-0.54,1.66,14.48,XY24,NTT,PAINTS
71,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.44,108766.0,-23069.0,65292.0,-0.16,-17.50,60.03,32.03,98.0,-0.35,0.80,21.83,OX40N,NTT,IT
18,CERA,9475.0,-21.00,40.0,H-SC,2.14,143922.0,-31981.0,74005.0,-1.42,-18.18,51.42,23.89,149.0,-0.43,1.05,24.30,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,71.0,M-MC,7.57,234536.0,9574.0,158077.0,2.27,4.26,67.40,74.53,192.0,0.06,1.72,37.88,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.49,196340.0,31670.0,11663.0,0.12,19.23,5.94,26.31,89.0,2.72,1.44,28.75,X40N,NTT,AC
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,213045.0,21303.0,16447.0,-0.41,11.11,7.72,19.69,99.0,1.30,1.56,18.66,XY25,NTT,AC
17,CAMS,5211.76,1.04,68.0,H-SC,3.44,114960.0,12956.0,30970.0,0.75,12.70,26.94,43.06,122.0,0.42,0.84,31.99,X40N,ATH,MISC
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.91,248087.0,356.0,74699.0,-0.25,0.14,30.11,30.30,92.0,0.00,1.82,13.45,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,7.96,112434.0,11930.0,17180.0,0.40,11.87,15.28,28.96,79.0,0.69,0.82,49.87,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.41,52.0,M-SC,0.66,100725.0,9226.0,69520.0,0.57,10.08,69.02,86.07,223.0,0.13,0.74,46.27,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,59.0,M-LC,5.66,158776.0,7831.0,101617.0,-0.33,5.19,64.00,72.51,53.0,0.08,1.16,11.52,XR,NTT,IT
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130762.0,7426.0,126342.0,-2.21,6.02,96.62,108.45,119.0,0.06,0.96,29.72,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.34,185483.0,5675.0,106745.0,1.34,3.16,57.55,62.52,88.0,0.05,1.36,38.32,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302603.0,-13239.0,125913.0,0.16,-4.19,41.61,35.67,5.0,-0.11,2.21,9.08,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.75,218196.0,-33572.0,86777.0,0.03,-13.33,39.77,21.13,27.0,-0.39,1.60,17.37,X40,ATH,PAINTS
30,HCLTECH,1943.91,1.08,48.0,H-LC,9.74,230523.0,-11401.0,74666.0,-1.71,-4.71,32.39,26.15,8.0,-0.15,1.69,8.93,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200857.0,719.0,20166.0,-0.19,0.36,10.04,10.44,4.0,0.04,1.47,5.94,X40,NTT,FMCG
79,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.76,228622.0,-5522.0,54138.0,0.14,-2.36,23.68,20.77,86.0,-0.10,1.67,4.40,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-0.29,68.0,H-LC,1.47,247545.0,1394.0,77803.0,-0.43,0.57,31.43,32.18,16.0,0.02,1.81,36.81,X200,ATH,IT
41,ITC,452.0,-37.92,50.0,H-LC,2.37,200857.0,719.0,20166.0,-0.19,0.36,10.04,10.44,4.0,0.04,1.47,5.94,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.46,279630.0,14204.0,42588.0,-1.17,5.35,15.23,21.40,11.0,0.33,2.05,13.55,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,56.0,H-LC,4.95,219632.0,9266.0,19525.0,-0.50,4.40,8.89,13.68,37.0,0.47,1.61,21.66,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.09,56.0,H-LC,5.69,215994.0,11494.0,29915.0,-0.10,5.62,13.85,20.25,10.0,0.38,1.58,14.84,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.76,228622.0,-5522.0,54138.0,0.14,-2.36,23.68,20.77,86.0,-0.10,1.67,4.40,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.15,59.0,H-LC,12.29,298055.0,-47901.0,122769.0,-0.19,-13.85,41.19,21.64,1.0,-0.39,2.18,5.59,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200857.0,719.0,20166.0,-0.19,0.36,10.04,10.44,4.0,0.04,1.47,5.94,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,48.0,H-LC,9.74,230523.0,-11401.0,74666.0,-1.71,-4.71,32.39,26.15,8.0,-0.15,1.69,8.93,X40,ATH,IT
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302603.0,-13239.0,125913.0,0.16,-4.19,41.61,35.67,5.0,-0.11,2.21,9.08,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.38,64.0,H-LC,26.99,166250.0,-19845.0,232235.0,0.22,-10.66,139.69,114.13,15.0,-0.09,1.22,22.43,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200857.0,719.0,20166.0,-0.19,0.36,10.04,10.44,4.0,0.04,1.47,5.94,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,56.0,H-LC,5.69,215994.0,11494.0,29915.0,-0.10,5.62,13.85,20.25,10.0,0.38,1.58,14.84,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.75,218196.0,-33572.0,86777.0,0.03,-13.33,39.77,21.13,27.0,-0.39,1.60,17.37,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219632.0,9266.0,19525.0,-0.50,4.40,8.89,13.68,37.0,0.47,1.61,21.66,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,59.0,H-LC,12.29,298055.0,-47901.0,122769.0,-0.19,-13.85,41.19,21.64,1.0,-0.39,2.18,5.59,X40,ATH,IT
39,INFY,2275.00,-15.65,59.0,H-LC,8.14,328616.0,15728.0,155961.0,0.14,5.03,47.46,54.87,3.0,0.10,2.40,12.01,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200857.0,719.0,20166.0,-0.19,0.36,10.04,10.44,4.0,0.04,1.47,5.94,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302603.0,-13239.0,125913.0,0.16,-4.19,41.61,35.67,5.0,-0.11,2.21,9.08,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,260966.0,-653.0,119862.0,0.01,-0.25,45.93,45.57,7.0,-0.01,1.91,11.04,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7005.56,65.0,L-SC,5.20,77937.0,-15873.0,92496.0,0.48,-16.92,118.68,81.67,269.0,-0.17,0.57,50.93,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,52.0,L-SC,42.25,84864.0,-28685.0,68740.0,0.55,-25.26,81.00,35.27,268.0,-0.42,0.62,107.71,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,85968.0,-16242.0,109635.0,2.94,-15.89,127.53,91.37,208.0,-0.15,0.63,39.55,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.51,47.0,H-SC,12.49,88825.0,-12138.0,106670.0,-0.46,-12.02,120.09,93.63,148.0,-0.11,0.65,32.30,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94515.0,-3465.0,25065.0,0.30,-3.54,26.52,22.05,152.0,-0.14,0.69,37.38,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1123.29,71.0,M-SC,4.66,166382.0,-14761.0,60230.0,2.03,-8.15,36.20,25.10,235.0,-0.25,1.22,33.34,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-14.58,78.0,H-SC,12.46,228613.0,-42148.0,372091.0,6.79,-15.57,162.76,121.86,133.0,-0.11,1.67,78.84,AR,ATH,IT
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,85968.0,-16242.0,109635.0,2.94,-15.89,127.53,91.37,208.0,-0.15,0.63,39.55,XY25,NTT,FINANCE
74,TITAGARH,1548.00,2.06,70.0,H-SC,3.14,167326.0,-24689.0,103575.0,1.02,-12.86,61.90,41.08,158.0,-0.24,1.22,41.79,XY24,NTT,ENGINEERING
7,ATULAUTO,844.00,3797.30,59.0,M-SC,5.38,121446.0,-25922.0,77738.0,-2.57,-17.59,64.01,35.16,236.0,-0.33,0.89,24.04,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.05
1,25,44.01
2,50,74.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.55
LC    32.12
MC    23.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.33
X40      15.43
X40N     11.93
XR       11.79
AR        8.85
XY25      8.25
OX40N     8.19
X200      1.81
MH        1.71
X5K       1.44
SR        1.28
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.30
H-LC    25.40
H-MC    20.31
M-SC    13.67
M-LC     5.65
M-MC     2.72
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.73,-11.97,73.02
FMCG,11.89,-4.10,40.53
FINANCE,9.76,-15.30,60.46
MISC,7.43,-13.26,76.22
BANKS,6.39,-14.44,74.64
PAINTS,5.85,-15.55,32.77
ELECTRICAL,5.77,-4.64,62.30
AC,3.72,4.69,11.26
INSURANCE,3.43,-7.10,43.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2838135.0,22
XR,1298257.0,14
AR,1287981.0,9
X40,740010.0,10
X40N,573060.0,9
OX40N,527203.0,10
XY25,337891.0,6
SR,246313.0,2
X5K,120519.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3405915.0,29
M-SC,1379461.0,17
H-LC,1222179.0,14
H-MC,1092911.0,15
M-LC,377553.0,4
M-MC,305961.0,2
L-SC,244840.0,3
L-MC,60098.0,1
L-LC,38194.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1023735.0      6
M-SC       XY24       798948.0      7
H-SC       AR         774431.0      5
           XR         754955.0      7
H-MC       XY24       494213.0      4
H-LC       X40        460339.0      5
           AR         352097.0      2
M-MC       XY24       305961.0      2
H-SC       X40N       290005.0      4
           OX40N      253716.0      4
           SR         246313.0      2
H-LC       X40N       209966.0      3
H-MC       X40        208443.0      4
M-SC       AR         161453.0      2
L-SC       XR         161236.0      2
M-LC       XY24       155417.0      2
H-MC       XY25       127949.0      2
M-SC       OX40N      124591.0      4
M-LC       X5K        120519.0      1
M-SC       XR         113606.0      2
           XY25       109635.0      1
H-MC       XR         106745.0      1
M-LC       XR         101617.0      1
L-SC       OX40N       83604.0      1
H-LC       X200        77803.0      1
H-MC       X40N        73089.0      2
M-SC       X40         71228.0      1
H-MC       OX40N       65292.0      1
H-SC       MH          62760.0      1
H-LC       XY25        62113.0      2
L-MC       XR          60098.0      1
H-LC       XY24        59861.0      1
L-LC       XY25        38194.0      1
H-MC       MH          17180.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
